<a href="https://colab.research.google.com/github/csh-scl/LLM-Study/blob/main/PromptTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai langchain langchain-openai
!pip install langchain_community

In [ ]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

#기본 LLM 로드를 위한 라이브러리 호출
from langchain.llms import OpenAI

#채팅 LLM 로드를 위한 라이브러리 호출
from langchain.chat_models import ChatOpenAI

In [ ]:
#OPENAI API키 저장
os.environ["OPENAI_API_KEY"] = 'Your API Key'

-------

#### Davinch-003 모델 설정하기

In [ ]:
davinch3 = OpenAI(
    model_name="text-davinci-003",
    max_tokens = 1000
)

<ipython-input-9-349a1428bdbb>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  davinch3 = OpenAI(


#### (1) 프롬프트 템플릿 맛보기

프롬프트 템플릿은 크게 2가지가 존재합니다.
1. Prompt Template
2. Chat Prompt Template

1번 Prompt Template은 일반적인 프롬프트 템플릿을 생성할때 활용합니다.

2번 Chat Prompt Template은 채팅 LLM에 프롬프트를 전달하는 데에 활용할 수 있는 특화 프롬프트 템플릿입니다.



In [ ]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

#프롬프트 템플릿을 통해 매개변수 삽입 가능한 문자열로 변환
string_prompt = PromptTemplate.from_template("tell me a joke about {subject} {price}")

#매개변수 삽입한 결과를 string_prompt_value에 할당
string_prompt_value = string_prompt.format_prompt(subject="soccer", price = 10000)

#채팅LLM이 아닌 LLM과 대화할 때 필요한 프롬프트 = string prompt
string_prompt_value

StringPromptValue(text='tell me a joke about soccer 10000')

In [ ]:
#to_string() 함수를 통해 prompt template으로 생성한 문장 raw_text 반환 가능
print(string_prompt_value.to_string())

tell me a joke about soccer


In [ ]:
chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}")
chat_prompt_value = chat_prompt.format_prompt(subject="soccer")
chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about soccer', additional_kwargs={}, response_metadata={})])

In [ ]:
from langchain_community.llms import OpenAI
from langchain.prompts import PromptTemplate

# 프롬프트 템플릿 생성
string_prompt = PromptTemplate.from_template("Could you recommend {subject} nearby {price}?")
string_prompt_value = string_prompt.format_prompt(subject= "food", price=10000)

# OpenAI LLM 생성 (API 키는 환경변수에서 자동으로 인식)
llm = OpenAI()

# 프롬프트 실행
response = llm.invoke(string_prompt_value.to_string())

print(response)


NameError: name 'food' is not defined

In [ ]:
chat_prompt_value.to_string()

'Human: tell me a joke about soccer'

#### (2) 프롬프트 템플릿 활용해보기

반복적인 프롬프트를 삽입해야하는 경우, Prompt Template를 통해 간편하게 LLM을 활용할 수 있습니다.

- GPT-3와 프롬프트 템플릿을 활용하여 대화해보기

In [ ]:
from langchain.prompts.prompt import PromptTemplate

template = """
너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
{재료}{수량}개

"""



In [ ]:
print(prompt_template.format(재료 = '쌀, 치킨, 너구리'))


너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
쌀, 치킨, 너구리




In [ ]:
print(davinch3(
    prompt_template.format(
        재료 = '양파, 계란, 사과, 빵'
    )
))

NameError: name 'prompt_template' is not defined

-  ChatGPT와 프롬프트 템플릿을 활용하여 대화해보기

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# ✅ 템플릿 수정: 재료목록만 받음
template = """
너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
{재료목록}
"""

chatgpt = ChatOpenAI(temperature=0)

system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{재료목록}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# ✅ 재료별 수량 조합 후 문자열로 변환
재료들 = ["양파", "계란", "사과", "빵"]
수량들 = [1, 2, 2, 1]

재료목록 = "\n".join(f"{재료} {수량}개" for 재료, 수량 in zip(재료들, 수량들))

# ✅ 포맷팅해서 출력
#print(chat_prompt.format_prompt(재료목록=재료목록).to_messages())

print(재료목록)

answer = chatgpt(chat_prompt.format_prompt(재료목록=재료목록).to_messages())
print(answer)
#print(answer.content)


양파 1개
계란 2개
사과 2개
빵 1개
가지고 계신 재료로는 "과일 샐러드와 계란 토스트"를 만들어보시는 건 어떨까요? 

**과일 샐러드와 계란 토스트 레시피:**

**과일 샐러드 재료:**
- 사과 1개 (나머지 1개는 계란 토스트에 활용)
- 양파 1/4개
- 빵 1/2개

**과일 샐러드 만드는 법:**
1. 먼저 양파를 얇게 채 썰어 준비합니다.
2. 한 개의 사과를 껍질을 벗기고 작은 조각으로 썰어줍니다.
3. 그 다음, 준비한 양파와 사과를 볼에 넣고 섞어줍니다.

**계란 토스트 재료:**
- 계란 2개
- 사과 1개
- 빵 1/2개

**계란 토스트 만드는 법:**
1. 계란 2개를 그릇에 넣고 잘 풀어줍니다.
2. 팬에 기름을 두르고 계란을 부어 계란프라이를 만들어줍니다.
3. 빵을 토스터에 구워줍니다.
4. 구운 빵 위에 계란프라이와 양파, 사과를 얹어 완성합니다.

과일 샐러드와 계란 토스트를 함께 즐기면 건강하고 맛있는 한 끼 식사가 완성될 것입니다. 맛있게 드세요! 🍳🍞🍎🥗


###(3) Few-shot 예제를 통한 프롬프트 템플릿

Few-shot이란, 딥러닝 모델이 결과물을 출력할 때 예시 결과물을 제시함으로써 원하는 결과물로 유도하는 방법론입니다.

LLM 역시, Few-shot 예제를 제공하면 예제와 유사한 형태의 결과물을 출력합니다.

내가 원하는 결과물의 형태가 특수하거나, 구조화된 답변을 원할 경우, 결과물의 예시를 수 개 제시함으로써 결과물의 품질을 향상시킬 수 있습니다.

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "아이유로 삼행시 만들어줘",
    "answer":
"""
아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.
"""
  },

  {
    "question": "김민수로 삼행시 만들어줘",
    "answer":
"""
김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!
"""
  }
]

In [ ]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.



In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="호날두로 삼행시 만들어줘"))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.


Question: 김민수로 삼행시 만들어줘

김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!


Question: 호날두로 삼행시 만들어줘


In [ ]:
from langchain.llms import OpenAI

model = OpenAI()  # 또는 적절한 모델 이름
print(model.predict("호날두로 삼행시 만들어줘"))



아하 명령어 센세이 올리고

무엇이든지

로나우도

빛나는 별처럼

축구를 사랑하는 열정이

우리 모두에게 넘치는 나라

우리 나라의 영웅이로다

날렵한 발로 세상을 누비는

오직 하나뿐인 존재

남다른 패기와 끈기로

저 나라를 대표하는

크리스티아누 로나우도

지구 상의 모든 축구대회에서

매번 우리 눈과 마음을 사로잡는

축구의 달인이로다

영원히 빛나는 별처럼

우리는 그를 사랑하며

늘 그의 무대 위를 응원할 것이다

크리스티아누 로나우도


In [ ]:
print(model(
    prompt.format(input="호날두로 삼행시 만들어줘")
))



호: 호떡도 좋지만
날: 날씨가 따뜻해지면
두: 두부까지 먹고 싶은 날이다.


### (4) Example Selector를 이용한 동적 Few-shot 러닝

Few-shot 예제를 동적으로 입력하고 싶은 경우, Example Selector를 활용할 수 있습니다.

LLM이 여러 작업을 수행하도록 만들되 내가 원하는 범위의 대답을 출력하도록 하려면 사용자의 입력에 동적으로 반응해야 합니다.이와 동시에, 예제를 모두 학습시키는 것이 아니라 적절한 예시만 포함하도록 함으로써 입력 prompt의 길이를 제한하고, 이를 통해 오류가 발생하지 않도록 조절할 수 있습니다.

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# These are a lot of examples of a pretend task of creating antonyms.
examples = [
    {"input": "행복", "output": "슬픔"},
    {"input": "흥미", "output": "지루"},
    {"input": "불안", "output": "안정"},
    {"input": "긴 기차", "output": "짧은 기차"},
    {"input": "큰 공", "output": "작은 공"},
]

ModuleNotFoundError: Module langchain_community.vectorstores not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [ ]:
!pip install chromadb
!pip install tiktoken

In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘",
    suffix="Input: {단어}\nOutput:",
    input_variables=["단어"],
)

In [ ]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(단어="무서운"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 불안
Output: 안정

Input: 무서운
Output:


In [ ]:
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(단어="큰 비행기"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 긴 기차
Output: 짧은 기차

Input: 큰 비행기
Output:


In [ ]:
query = "큰 비행기"

print(davinch3(
    similar_prompt.format(단어=query)
))

 작은 비행기


### (5) Output Parser를 활용한 출력값 조정

LLM의 답변을 내가 원하는 형태로 고정하고 싶다면 OutputParser 함수를 활용할 수 있습니다. 리스트, JSON 형태 등 다양한 형식의 답변을 고정하여 출력할 수 있습니다.

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [ ]:
output_parser = CommaSeparatedListOutputParser()

In [ ]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [ ]:
prompt = PromptTemplate(
    template="{주제} 5개를 추천해줘.\n{format_instructions}",
    input_variables=["주제"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
model = OpenAI(temperature=0)

In [ ]:
_input = prompt.format(주제="영화")
output = model(_input)

In [ ]:
output

'\n\n아이 스타일, 스타 워즈: 깨어난 포스, 엔드게임, 스파이더맨: 홈커밍, 엑스맨: 데이즈 오브 퓨처, 어벤져스: 엔드게임'

In [ ]:
output_parser.parse(output)

['아이 스타일',
 '스타 워즈: 깨어난 포스',
 '엔드게임',
 '스파이더맨: 홈커밍',
 '엑스맨: 데이즈 오브 퓨처',
 '어벤져스: 엔드게임']